In [28]:
import pandas as pd
import numpy as np

tickers = pd.read_csv('data/spy_holdings_24.csv')

In [29]:
# compute absolute returns (daily)

for ti in range(len(tickers)):
    tick = tickers.iloc[ti]['Ticker']

    dsi = pd.read_csv(f'data/{tick}.csv')
    if len(dsi) == 0:
        continue

    lags = [1,5,20,60,250,500,1000]
    maxlag = 1000

    M = len(dsi)

    cols = ['Date','Ticker','Ret']

    data = []
        
    for j in range(len(lags)):
        cols.append(f'Ret{lags[j]}D')

    for i in range(maxlag+1, M):
        date = dsi.iloc[i]['Date']
        if int(date[0:4]) < 2000:
            continue
        row = []
        row.append(date)
        row.append(dsi.iloc[i]['Ticker'])
        
        px = dsi.iloc[i]['Adj Close']
        pxp = dsi.iloc[i-1]['Adj Close']
        ret = np.log(px/pxp)
        row.append(ret)
        
        for j in range(len(lags)):
            lag = lags[j]
            pxlb = dsi.iloc[i-lag-1]['Adj Close']
            ret = np.log(pxp/pxlb)/lag
            row.append(ret)
        data.append(row)

    df = pd.DataFrame(data,columns=cols)
    df.to_csv(f'data/RET_{tick}.csv')






In [30]:
# combined absolute returns

for y in range(2000,2025):

    dfs = []

    for ti in range(len(tickers)):
        tick = tickers.iloc[ti]['Ticker']
        try:
            dfi = pd.read_csv(f'data/RET_{tick}.csv')
        except:
            continue
        dfi['Year'] = dfi['Date'].apply(lambda x: int(x[0:4]))
        dfi = dfi[dfi['Year'] == y]
        dfs.append(dfi)

    df = pd.concat(dfs)

    dff = df[['Ret','Ret1D','Ret5D','Ret20D','Ret60D','Ret250D','Ret500D','Ret1000D']]
    dff.reset_index(drop=True,inplace=True)
    dff.head()

    dff.to_csv(f'data/{y}_RET.csv')
 


/tmp/ipykernel_57972/1927531446.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs)
/tmp/ipykernel_57972/1927531446.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs)
/tmp/ipykernel_57972/1927531446.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

In [40]:
dfspy = pd.read_csv('data/SPY.csv')
dfspy.head()

lags = [1,5,20,60,250,500,1000]
maxlag = 1000

M = len(dfspy)

spy_rets = {}

for i in range(maxlag+1, M):
    date = dfspy.iloc[i]['Date']
    if int(date[0:4]) < 2000:
        continue
    row = []
    
    px = dfspy.iloc[i]['Adj Close']
    pxp = dfspy.iloc[i-1]['Adj Close']
    ret = np.log(px/pxp)
    row.append(ret)
    
    for j in range(len(lags)):
        lag = lags[j]
        pxlb = dfspy.iloc[i-lag-1]['Adj Close']
        ret = np.log(pxp/pxlb)/lag
        row.append(ret)
    spy_rets[date] = row

#print(spy_rets)

for y in range(2000,2025):

    dfs = []

    for ti in range(len(tickers)):
        tick = tickers.iloc[ti]['Ticker']
        try:
            dfi = pd.read_csv(f'data/RET_{tick}.csv')
        except:
            continue
        dfi['Year'] = dfi['Date'].apply(lambda x: int(x[0:4]))
        dfi = dfi[dfi['Year'] == y]
        
        for i in range(len(dfi)):
            row = spy_rets[dfi.iloc[i]['Date']]
            for j in range(len(lags)+1):
                #print(dfi.iloc[i,2+j])
                dfi.iloc[i,3+j] = dfi.iloc[i,3+j] - row[j]
        
        dfs.append(dfi)

    df = pd.concat(dfs)

    dff = df[['Ret','Ret1D','Ret5D','Ret20D','Ret60D','Ret250D','Ret500D','Ret1000D']]
    dff.reset_index(drop=True,inplace=True)
    dff.head()

    dff.to_csv(f'data/{y}_RELRET.csv')

AAPL


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U4'), dtype('float64')) -> None